In [16]:
from ollama import Client, Options
from typing import Dict, Any, List
import yaml
import re
import os
from torch import Tensor
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score

tqdm.pandas()

CHECKWORTHY_PROMPT = """Given a sentence from a political debate or tweet, your task is to predict if it is fact check-worthy statement or not.
 Your response must always start with "My response is: (yes or no based on your prediction)"
 your prediction must be yes or no, and i wont accept you prediction to be "I cannot determine"
 Never write "I'm and AI language model..."
 Examples:
 fact check-worthy: The sky is blue.
 Not fact check-worthy: I like ice cream.
 fact check-worthy: The distance between the earth and the moon is 238,855 km.
 Not fact check-worthy: It made the best ratings in the history of television.
 fact check-worthy: The unemployment rate is 5.2% in Norway.
 Not fact check-worthy: I am the best president in the history of the United States.
Sentence: {claim}"""

In [17]:
class Ollama:
    """A class for generating questions and interacting with the Ollama API."""

    def __init__(self, config_path: str = "mistral.yaml"):
        """Initializes the Ollama client and loads necessary configurations."""
        self._ollama_client = Client(timeout=20)
        self._config_path = config_path
        self._config = self._load_config()
        self._stream = self._config.get("stream", False)
        self._model_name = self._config.get("model", "mistral")
        self._llm_options = self._get_llm_config()

    def generate(self, prompt: str) -> str:
        """Generate text using Ollama LLM for the given prompt.

        Args:
            prompt: Prompt for the LLM.

        Returns:
            Response text from an Ollama LLM.
        """
        response = self._ollama_client.generate(
            model=self._model_name,
            prompt=prompt,
            options=self._llm_options,
            stream=self._stream,
        )
        return response.get("response", "").strip()  # type: ignore

    def _load_config(self) -> Dict[str, Any]:
        """Loads configuration from a YAML file.

        Raises:
            FileNotFoundError: If the config file is not found.

        Returns:
            A dictionary with configuration values.
        """
        if not os.path.isfile(self._config_path):
            raise FileNotFoundError(f"Config file {self._config_path} not found.")
        with open(self._config_path, "r") as file:
            yaml_data = yaml.safe_load(file)
        return yaml_data

    def _get_llm_config(self) -> Options:
        """Extracts and returns the LLM (language learning model) configuration.

        Returns:
            An Options object with the LLM configuration.
        """
        return Options(self._config.get("options", {}))

In [18]:
ollama = Ollama()

In [19]:
listofpredicts = list()
df = pd.read_csv(
    "../data/processed/processed_CT24_checkworthy_english/processed_dev_test.tsv",
    sep="\t",
)
for index, row in tqdm(df.iterrows(), total=len(df)):
    response = ollama.generate(CHECKWORTHY_PROMPT.format(claim=row["text"]))
    response_words = response.split()
    answer = response_words[3].strip(".,").lower()
    if answer == "yes" or answer == "no":
        listofpredicts.append(response_words[3].strip(".,").lower())
    else:
        listofpredicts.append("no")

100%|██████████| 318/318 [20:24<00:00,  3.85s/it]   


In [20]:
print(listofpredicts)

['no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no

In [21]:
f1 = f1_score(df["class_label"].str.lower(), listofpredicts, average="weighted")
print("F1 Score:", f1)

F1 Score: 0.8450755940916146
